In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
def load_and_preprocess(filepath, is_test=False):
    data = pd.read_csv(filepath, delimiter=' ::: ', engine='python', header=None)
    if is_test:
        data.columns = ['ID', 'TITLE', 'DESCRIPTION']
    else:
        data.columns = ['ID', 'TITLE', 'GENRE', 'DESCRIPTION']
    data['DESCRIPTION'] = data['DESCRIPTION'].str.lower().str.replace('[^\w\s]', '', regex=True)
    return data

In [3]:
train_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt')
test_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt', is_test=True)
test_solution_data = load_and_preprocess('/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt')

In [4]:
X_train = train_data['DESCRIPTION']
y_train = train_data['GENRE']
X_test = test_data['DESCRIPTION']
y_test = test_solution_data['GENRE']

In [5]:
X_train

0        listening in to a conversation between his doc...
1        a brother and sister with a past incestuous re...
2        as the bus empties the students for their fiel...
3        to help their unemployed father make ends meet...
4        the films title refers not only to the unrecov...
                               ...                        
54209    this shortlived nbc live sitcom centered on bo...
54210    the next generation of exploitation the sister...
54211    ze bestaan echt is a standup comedy about grow...
54212    walter and vivian live in the country and have...
54213    on labor day weekend 1935 the most intense hur...
Name: DESCRIPTION, Length: 54214, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

In [8]:
y_pred = model.predict(X_test_tfidf)

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6189302362506517

In [10]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()